# Import libraries

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import pickle

Using TensorFlow backend.


# Load processed dataset
The data we load here are already shuffled and normalized.

In [2]:
data_directory = 'dataset/7. oxford-normalized-10/'
x_train = pickle.load(open(data_directory + "x_train.p", "rb"))
y_train = pickle.load(open(data_directory + "y_train.p", "rb"))
x_test = pickle.load(open(data_directory + "x_test.p", "rb"))
y_test = pickle.load(open(data_directory + "y_test.p", "rb"))
print('x_train: ' + str(x_train.shape))
print('y_train: ' + str(y_train.shape))
print('x_test: ' + str(x_test.shape))
print('y_test: ' + str(y_test.shape))

x_train: (175, 22)
y_train: (175, 2)
x_test: (20, 22)
y_test: (20, 2)


# Tunable parameters

In [3]:
epochs = 10000
batch_size = 20
learning_rate = 0.0001
decay = 0.0001
loss = keras.losses.mean_squared_error

# Autoencoder #1

In [4]:
date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
root_directory = 'results/51-oxford-autoencoder-1-10-01/'
model_directory = root_directory + "model/"
tensorboard_directory = root_directory + "tensorboard/"
def create_model(layer_size, dropout):
    global date
    x = Input(shape=(x_train.shape[1],))

    y = Dense(units = layer_size, activation='relu')(x)
    y = Dropout(dropout)(y)

    y = Dense(units = x_train.shape[1], activation='sigmoid')(y)
    model = Model(x, y)
    
    # Create directory
    directory = model_directory + date + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Write model hyper-parameters
    df = pd.DataFrame({
        'Hidden layer units': layer_size,
        'Dropout': dropout,
        'Learning Rate': learning_rate,
        'Decay': decay,
        'Epochs': epochs,
        'Batch Size': batch_size,
        'Loss': loss
    }, index=[0])
    df.to_csv(model_directory + date + '/params.csv')
    
    # Write model summary
    file2 = open(directory + "summary.txt", "a")
    model.summary(print_fn=lambda line: file2.write(line + '\n'))
    file2.close()

    # Write model diagram
    plot_model(model, to_file=directory + 'model.png', show_shapes=True, show_layer_names=False)
    
    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate, decay=decay), loss=loss, metrics=['accuracy'])
    
    return model

# Prepare callbacks

In [5]:
class KerasClassifierTensorBoard(KerasClassifier):
    def fit(self, x, y, **kwargs):
        global date
        date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_directory + date + '/')
        csv_logger = keras.callbacks.CSVLogger(model_directory + date + '/epochs.csv')
        model_checkpoint = keras.callbacks.ModelCheckpoint(model_directory + date + '/weights.hdf5', save_best_only=True)
        callbacks = [tensorboard_callback, csv_logger, model_checkpoint]
        return super(KerasClassifierTensorBoard, self).fit(x, y, callbacks=callbacks, **kwargs)

# Grid search

In [6]:
model = KerasClassifierTensorBoard(build_fn=create_model, epochs=epochs, batch_size=batch_size)

layers = [12, 8]
dropouts = [0.2, 0.4]
param_grid = dict(
    layer_size=layers, 
    dropout=dropouts
)

grid = GridSearchCV(estimator = model, param_grid = param_grid, error_score=0, verbose=2, cv=[(slice(None), slice(None))], n_jobs=1, fit_params=dict(validation_data=(x_test, x_test)))
grid_result = grid.fit(x_train, x_train)

Fitting 1 folds for each of 4 candidates, totalling 4 fits
[CV] layer_size=12, dropout=0 ........................................
Train on 175 samples, validate on 20 samples
Epoch 1/10000
175/175 [==============================] - 0s - loss: 0.1132 - acc: 0.0686 - val_loss: 0.1219 - val_acc: 0.1500
Epoch 2/10000
175/175 [==============================] - 0s - loss: 0.1126 - acc: 0.0686 - val_loss: 0.1214 - val_acc: 0.1500
Epoch 3/10000
175/175 [==============================] - 0s - loss: 0.1121 - acc: 0.0686 - val_loss: 0.1208 - val_acc: 0.1500
Epoch 4/10000
175/175 [==============================] - 0s - loss: 0.1115 - acc: 0.0686 - val_loss: 0.1203 - val_acc: 0.1500
Epoch 5/10000
175/175 [==============================] - 0s - loss: 0.1110 - acc: 0.0686 - val_loss: 0.1198 - val_acc: 0.1500
Epoch 6/10000
175/175 [==============================] - 0s - loss: 0.1105 - acc: 0.0686 - val_loss: 0.1193 - val_acc: 0.1500
Epoch 7/10000
175/175 [==============================] - 0s - loss: 0

175/175 [==============================] - 0s - loss: 0.0815 - acc: 0.0686 - val_loss: 0.0924 - val_acc: 0.1500
Epoch 65/10000
175/175 [==============================] - 0s - loss: 0.0810 - acc: 0.0686 - val_loss: 0.0919 - val_acc: 0.1500
Epoch 66/10000
175/175 [==============================] - 0s - loss: 0.0805 - acc: 0.0686 - val_loss: 0.0915 - val_acc: 0.1500
Epoch 67/10000
175/175 [==============================] - 0s - loss: 0.0800 - acc: 0.0686 - val_loss: 0.0910 - val_acc: 0.1500
Epoch 68/10000
175/175 [==============================] - 0s - loss: 0.0795 - acc: 0.0686 - val_loss: 0.0906 - val_acc: 0.1500
Epoch 69/10000
175/175 [==============================] - 0s - loss: 0.0790 - acc: 0.0686 - val_loss: 0.0901 - val_acc: 0.1500
Epoch 70/10000
175/175 [==============================] - 0s - loss: 0.0785 - acc: 0.0686 - val_loss: 0.0897 - val_acc: 0.1500
Epoch 71/10000
175/175 [==============================] - 0s - loss: 0.0780 - acc: 0.0686 - val_loss: 0.0893 - val_acc: 0.1500

175/175 [==============================] - 0s - loss: 0.0567 - acc: 0.0686 - val_loss: 0.0699 - val_acc: 0.1500
Epoch 129/10000
175/175 [==============================] - 0s - loss: 0.0564 - acc: 0.0686 - val_loss: 0.0696 - val_acc: 0.1500
Epoch 130/10000
175/175 [==============================] - 0s - loss: 0.0561 - acc: 0.0743 - val_loss: 0.0694 - val_acc: 0.1500
Epoch 131/10000
175/175 [==============================] - 0s - loss: 0.0558 - acc: 0.0800 - val_loss: 0.0691 - val_acc: 0.1500
Epoch 132/10000
175/175 [==============================] - 0s - loss: 0.0555 - acc: 0.0800 - val_loss: 0.0689 - val_acc: 0.1500
Epoch 133/10000
175/175 [==============================] - 0s - loss: 0.0553 - acc: 0.0800 - val_loss: 0.0686 - val_acc: 0.1500
Epoch 134/10000
175/175 [==============================] - 0s - loss: 0.0550 - acc: 0.0800 - val_loss: 0.0683 - val_acc: 0.1500
Epoch 135/10000
175/175 [==============================] - 0s - loss: 0.0547 - acc: 0.0971 - val_loss: 0.0681 - val_acc:

175/175 [==============================] - 0s - loss: 0.0430 - acc: 0.4171 - val_loss: 0.0567 - val_acc: 0.2500
Epoch 193/10000
175/175 [==============================] - 0s - loss: 0.0429 - acc: 0.4171 - val_loss: 0.0565 - val_acc: 0.2500
Epoch 194/10000
175/175 [==============================] - 0s - loss: 0.0427 - acc: 0.4343 - val_loss: 0.0564 - val_acc: 0.2500
Epoch 195/10000
175/175 [==============================] - 0s - loss: 0.0426 - acc: 0.4343 - val_loss: 0.0562 - val_acc: 0.2500
Epoch 196/10000
175/175 [==============================] - 0s - loss: 0.0424 - acc: 0.4343 - val_loss: 0.0561 - val_acc: 0.2500
Epoch 197/10000
175/175 [==============================] - 0s - loss: 0.0422 - acc: 0.4343 - val_loss: 0.0559 - val_acc: 0.2500
Epoch 198/10000
175/175 [==============================] - 0s - loss: 0.0421 - acc: 0.4343 - val_loss: 0.0558 - val_acc: 0.2500
Epoch 199/10000
175/175 [==============================] - 0s - loss: 0.0419 - acc: 0.4457 - val_loss: 0.0556 - val_acc:

175/175 [==============================] - 0s - loss: 0.0352 - acc: 0.4800 - val_loss: 0.0484 - val_acc: 0.2000
Epoch 257/10000
175/175 [==============================] - 0s - loss: 0.0351 - acc: 0.4743 - val_loss: 0.0483 - val_acc: 0.2000
Epoch 258/10000
175/175 [==============================] - 0s - loss: 0.0350 - acc: 0.4800 - val_loss: 0.0482 - val_acc: 0.2000
Epoch 259/10000
175/175 [==============================] - 0s - loss: 0.0349 - acc: 0.4800 - val_loss: 0.0481 - val_acc: 0.2000
Epoch 260/10000
175/175 [==============================] - 0s - loss: 0.0348 - acc: 0.4743 - val_loss: 0.0480 - val_acc: 0.2000
Epoch 261/10000
175/175 [==============================] - 0s - loss: 0.0347 - acc: 0.4743 - val_loss: 0.0479 - val_acc: 0.2000
Epoch 262/10000
175/175 [==============================] - 0s - loss: 0.0346 - acc: 0.4743 - val_loss: 0.0478 - val_acc: 0.2000
Epoch 263/10000
175/175 [==============================] - 0s - loss: 0.0345 - acc: 0.4743 - val_loss: 0.0476 - val_acc:

...

175/175 [==============================] - 0s - loss: 0.0105 - acc: 0.5257 - val_loss: 0.0137 - val_acc: 0.3000
Epoch 1213/10000
175/175 [==============================] - 0s - loss: 0.0105 - acc: 0.5257 - val_loss: 0.0137 - val_acc: 0.3000
Epoch 1214/10000
175/175 [==============================] - 0s - loss: 0.0105 - acc: 0.5257 - val_loss: 0.0137 - val_acc: 0.3000
Epoch 1215/10000
175/175 [==============================] - 0s - loss: 0.0105 - acc: 0.5257 - val_loss: 0.0137 - val_acc: 0.3000
Epoch 1216/10000
175/175 [==============================] - 0s - loss: 0.0105 - acc: 0.5257 - val_loss: 0.0137 - val_acc: 0.3000
Epoch 1217/10000
175/175 [==============================] - 0s - loss: 0.0105 - acc: 0.5257 - val_loss: 0.0137 - val_acc: 0.3000
Epoch 1218/10000
175/175 [==============================] - 0s - loss: 0.0105 - acc: 0.5257 - val_loss: 0.0137 - val_acc: 0.3000
Epoch 1219/10000
175/175 [==============================] - 0s - loss: 0.0104 - acc: 0.5257 - val_loss: 0.0136 - v

175/175 [==============================] - 0s - loss: 0.0101 - acc: 0.5314 - val_loss: 0.0132 - val_acc: 0.3000
Epoch 1276/10000
175/175 [==============================] - 0s - loss: 0.0101 - acc: 0.5314 - val_loss: 0.0132 - val_acc: 0.3000
Epoch 1277/10000
175/175 [==============================] - 0s - loss: 0.0101 - acc: 0.5314 - val_loss: 0.0132 - val_acc: 0.3000
Epoch 1278/10000
175/175 [==============================] - 0s - loss: 0.0101 - acc: 0.5314 - val_loss: 0.0132 - val_acc: 0.3000
Epoch 1279/10000
175/175 [==============================] - 0s - loss: 0.0101 - acc: 0.5314 - val_loss: 0.0132 - val_acc: 0.3000
Epoch 1280/10000
175/175 [==============================] - 0s - loss: 0.0101 - acc: 0.5314 - val_loss: 0.0132 - val_acc: 0.3000
Epoch 1281/10000
175/175 [==============================] - 0s - loss: 0.0101 - acc: 0.5314 - val_loss: 0.0132 - val_acc: 0.3000
Epoch 1282/10000
175/175 [==============================] - 0s - loss: 0.0100 - acc: 0.5314 - val_loss: 0.0131 - v

175/175 [==============================] - 0s - loss: 0.0097 - acc: 0.5257 - val_loss: 0.0127 - val_acc: 0.3000
Epoch 1339/10000
175/175 [==============================] - 0s - loss: 0.0097 - acc: 0.5257 - val_loss: 0.0127 - val_acc: 0.3000
Epoch 1340/10000
175/175 [==============================] - 0s - loss: 0.0097 - acc: 0.5257 - val_loss: 0.0127 - val_acc: 0.3000
Epoch 1341/10000
175/175 [==============================] - 0s - loss: 0.0097 - acc: 0.5257 - val_loss: 0.0127 - val_acc: 0.3000
Epoch 1342/10000
175/175 [==============================] - 0s - loss: 0.0097 - acc: 0.5257 - val_loss: 0.0127 - val_acc: 0.3000
Epoch 1343/10000
175/175 [==============================] - 0s - loss: 0.0097 - acc: 0.5257 - val_loss: 0.0127 - val_acc: 0.3000
Epoch 1344/10000
175/175 [==============================] - 0s - loss: 0.0097 - acc: 0.5257 - val_loss: 0.0127 - val_acc: 0.3000
Epoch 1345/10000
175/175 [==============================] - 0s - loss: 0.0097 - acc: 0.5257 - val_loss: 0.0127 - v

175/175 [==============================] - 0s - loss: 0.0094 - acc: 0.5314 - val_loss: 0.0123 - val_acc: 0.3000
Epoch 1402/10000
175/175 [==============================] - 0s - loss: 0.0094 - acc: 0.5314 - val_loss: 0.0123 - val_acc: 0.3000
Epoch 1403/10000
175/175 [==============================] - 0s - loss: 0.0094 - acc: 0.5314 - val_loss: 0.0123 - val_acc: 0.3000
Epoch 1404/10000
175/175 [==============================] - 0s - loss: 0.0094 - acc: 0.5314 - val_loss: 0.0123 - val_acc: 0.3000
Epoch 1405/10000
175/175 [==============================] - 0s - loss: 0.0094 - acc: 0.5314 - val_loss: 0.0123 - val_acc: 0.3000
Epoch 1406/10000
175/175 [==============================] - 0s - loss: 0.0094 - acc: 0.5371 - val_loss: 0.0123 - val_acc: 0.3000
Epoch 1407/10000
175/175 [==============================] - 0s - loss: 0.0093 - acc: 0.5429 - val_loss: 0.0123 - val_acc: 0.3000
Epoch 1408/10000
175/175 [==============================] - 0s - loss: 0.0093 - acc: 0.5429 - val_loss: 0.0123 - v

KeyboardInterrupt: 